## Bias testing 
Choose one of the PREDICTANDS in the first cell and run entire cells to view the results.

In [1]:
# define the predictand
PREDICTAND = 'tasmean'  # 'tasmin' or 'tasmax' or 'tasmean'

In [2]:
# mapping from predictands to variable names
NAMES = {'tasmin': 'minimum temperature', 'tasmax': 'maximum temperature', 'tasmean': 'mean temperature'}
# dataset names mapping
PRED = 'LGBMRegression'
TRUE = 'CERRA_tasmean'

### Imports

In [3]:
# builtins
import datetime
import warnings
import calendar

# externals
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import scipy.stats as stats
from IPython.display import Image
from sklearn.metrics import r2_score

### Load datasets

In [4]:
if PREDICTAND == 'tasmin':
    y_pred = xr.open_dataset("/mnt/CEPH_PROJECTS/InterTwin/Climate_Downscaling/preliminary_result/CERRA/tasmin/2m_daily_min_temperature_1985_to_2016.nc")
    
if PREDICTAND == 'tasmax':
    y_pred = xr.open_dataset("/mnt/CEPH_PROJECTS/InterTwin/Climate_Downscaling/preliminary_result/CERRA/tasmax/2m_daily_max_temperature_1985_to_2016.nc")
    
if PREDICTAND == 'tasmean':
    y_pred = xr.open_dataset("/mnt/CEPH_PROJECTS/InterTwin/Climate_Downscaling/larger_alps/ml_Predictions/tasmean/LGBMRegressor_tasmean_ztuvq_500_850_mslpt2_0mm_dem_sa_doy.nc")    

y_pred


<xarray.Dataset>
Dimensions:  (time: 212, number: 25, y: 103, x: 125)
Coordinates:
  * time     (time) datetime64[ns] 2016-01-02 2016-01-03 ... 2016-07-31
  * number   (number) int32 0 1 2 3 4 5 6 7 8 9 ... 16 17 18 19 20 21 22 23 24
  * y        (y) float64 50.52 50.45 50.37 50.3 50.23 ... 43.27 43.2 43.13 43.05
  * x        (x) float64 3.715 3.822 3.93 4.038 ... 16.75 16.85 16.96 17.07
Data variables:
    tasmean  (time, y, x, number) float64 ...

In [5]:
y_pred = y_pred - 273.15

In [6]:
# CERRA Data
if PREDICTAND == 'tasmin':
    y_true = xr.open_dataset("/mnt/CEPH_PROJECTS/InterTwin/Climate_Downscaling/preliminary_result/CERRA/tasmin/2m_daily_min_temperature_1985_to_2016.nc")
    
if PREDICTAND == 'tasmax':
    y_true = xr.open_dataset("/mnt/CEPH_PROJECTS/InterTwin/Climate_Downscaling/preliminary_result/CERRA/tasmax/2m_daily_max_temperature_1985_to_2016.nc")
    
if PREDICTAND == 'tasmean':
    y_true = xr.open_dataset("/mnt/CEPH_PROJECTS/InterTwin/Climate_Downscaling/larger_alps/CERRA/tasmean/CERRA_2m_temperature_1985_2020.nc")    


y_true

y_true = y_true.rename({'longitude': "x"})
y_true = y_true.rename({'latitude': "y"})
y_true

<xarray.Dataset>
Dimensions:            (x: 125, y: 103, time: 13149)
Coordinates:
  * x                  (x) float64 3.715 3.822 3.93 4.038 ... 16.85 16.96 17.07
  * y                  (y) float64 50.52 50.45 50.37 50.3 ... 43.2 43.13 43.05
    heightAboveGround  float64 ...
  * time               (time) datetime64[ns] 1985-01-01 ... 2020-12-31
Data variables:
    t2m                (time, y, x) float32 ...
Attributes:
    CDI:                     Climate Data Interface version 2.0.4 (https://mp...
    Conventions:             CF-1.7
    institution:             Norrkoping
    GRIB_centre:             eswi
    GRIB_centreDescription:  Norrkoping
    history:                 Sun Jul 09 20:20:26 2023: cdo remapbil,/mnt/CEPH...
    CDO:                     Climate Data Operators version 2.0.4 (https://mp...

In [7]:
# subset to time period covered by predictions
y_true = y_true.sel(time=y_pred.time)
y_true = y_true.rename({'t2m': PREDICTAND})
y_true = y_true - 273.15

In [8]:
y_true

<xarray.Dataset>
Dimensions:            (x: 125, y: 103, time: 212)
Coordinates:
  * x                  (x) float64 3.715 3.822 3.93 4.038 ... 16.85 16.96 17.07
  * y                  (y) float64 50.52 50.45 50.37 50.3 ... 43.2 43.13 43.05
    heightAboveGround  float64 ...
  * time               (time) datetime64[ns] 2016-01-02 ... 2016-07-31
Data variables:
    tasmean            (time, y, x) float32 8.403 8.346 8.309 ... 26.52 26.97
Attributes:
    CDI:                     Climate Data Interface version 2.0.4 (https://mp...
    Conventions:             CF-1.7
    institution:             Norrkoping
    GRIB_centre:             eswi
    GRIB_centreDescription:  Norrkoping
    history:                 Sun Jul 09 20:20:26 2023: cdo remapbil,/mnt/CEPH...
    CDO:                     Climate Data Operators version 2.0.4 (https://mp...

In [9]:
y_pred

<xarray.Dataset>
Dimensions:  (time: 212, number: 25, y: 103, x: 125)
Coordinates:
  * time     (time) datetime64[ns] 2016-01-02 2016-01-03 ... 2016-07-31
  * number   (number) int32 0 1 2 3 4 5 6 7 8 9 ... 16 17 18 19 20 21 22 23 24
  * y        (y) float64 50.52 50.45 50.37 50.3 50.23 ... 43.27 43.2 43.13 43.05
  * x        (x) float64 3.715 3.822 3.93 4.038 ... 16.75 16.85 16.96 17.07
Data variables:
    tasmean  (time, y, x, number) float64 7.981 7.151 8.194 ... 24.34 20.87

In [10]:
def get_time_range(dataset):
    time_coord = dataset.time

    # Get the starting and ending years
    start_year = time_coord.min().dt.year.item()
    end_year = time_coord.max().dt.year.item()

    return str(start_year), str(end_year)

start_year, end_year = get_time_range(y_pred)
print("Starting year:", start_year)
print("Ending year:", end_year)


Starting year: 2016
Ending year: 2016


In [11]:
# align datasets and mask missing values in Insitu:CERRA predictions
y_true, y_pred = xr.align(y_true[PREDICTAND], y_pred[PREDICTAND], join='override')

In [12]:
y_true

<xarray.DataArray 'tasmean' (time: 212, y: 103, x: 125)>
array([[[  8.403107 ,   8.346222 ,   8.308624 , ...,  -7.0691223,
          -7.2671204,  -7.5451355],
        [  8.6692505,   8.613159 ,   8.562378 , ...,  -7.35318  ,
          -7.1090393,  -6.832947 ],
        [  8.386017 ,   8.023041 ,   8.061005 , ...,  -7.0759583,
          -7.492523 ,  -6.8048706],
        ...,
        [ 13.420258 ,  13.546875 ,  13.651337 , ...,  10.586761 ,
          10.632629 ,   9.359131 ],
        [ 13.460358 ,  13.566895 ,  13.681824 , ...,   8.985657 ,
           8.953613 ,   8.641937 ],
        [ 13.478363 ,  13.575775 ,  13.668579 , ...,  11.328552 ,
          10.776642 ,   9.357758 ]],

       [[  6.50412  ,   6.4684753,   6.4767456, ..., -11.638458 ,
         -11.618317 , -11.654724 ],
        [  6.704651 ,   6.676483 ,   6.696594 , ..., -11.883148 ,
         -11.638763 , -11.307587 ],
        [  6.4444885,   6.1457825,   6.2534485, ..., -11.593201 ,
         -11.912079 , -11.471893 ],
...
        [ 22.085785 ,  22.181244 ,  22.233002 , ...,  27.409332 ,
          27.288544 ,  27.110962 ],
        [ 22.14914  ,  22.217438 ,  22.262756 , ...,  27.823914 ,
          27.806    ,  27.89096  ],
        [ 22.237762 ,  22.277618 ,  22.290955 , ...,  25.957397 ,
          26.008087 ,  26.46698  ]],

       [[ 17.314636 ,  17.216858 ,  17.456696 , ...,  21.080322 ,
          21.06314  ,  21.085663 ],
        [ 17.554138 ,  17.911224 ,  18.093811 , ...,  20.422455 ,
          20.97815  ,  21.421173 ],
        [ 17.56012  ,  17.520782 ,  17.650665 , ...,  19.939575 ,
          19.552277 ,  20.77478  ],
        ...,
        [ 23.566742 ,  23.52835  ,  23.496857 , ...,  27.947388 ,
          27.895477 ,  28.06543  ],
        [ 23.402557 ,  23.466187 ,  23.451447 , ...,  28.208954 ,
          28.371796 ,  28.406006 ],
        [ 23.279755 ,  23.3078   ,  23.332367 , ...,  26.370941 ,
          26.518402 ,  26.972382 ]]], dtype=float32)
Coordinates:
  * x                  (x) float64 3.715 3.822 3.93 4.038 ... 16.85 16.96 17.07
  * y                  (y) float64 50.52 50.45 50.37 50.3 ... 43.2 43.13 43.05
    heightAboveGround  float64 ...
  * time               (time) datetime64[ns] 2016-01-02 ... 2016-07-31

In [13]:
y_pred

<xarray.DataArray 'tasmean' (time: 212, y: 103, x: 125, number: 25)>
array([[[[ 7.98058456,  7.15101325,  8.19358053, ...,  7.37668269,
           7.87068532,  7.46883532],
         [ 8.08372602,  7.26552142,  8.38391152, ...,  7.47442601,
           7.82020381,  7.47695811],
         [ 8.00990391,  7.33820502,  8.41876945, ...,  7.66024267,
           8.07426848,  7.76832782],
         ...,
         [-3.74700512, -4.36099742, -2.72102345, ..., -4.43398576,
          -2.38274903, -4.50441524],
         [-3.62221108, -4.17782213, -2.43768194, ..., -4.58282664,
          -2.17536966, -4.61320208],
         [-4.92410837, -5.05245441, -3.08960262, ..., -5.22946376,
          -2.74264454, -5.31873382]],

        [[ 8.37169314,  7.50378343,  8.60162555, ...,  7.75888534,
           8.20561951,  7.84688719],
         [ 8.23018033,  7.68747904,  8.57110293, ...,  7.76819135,
           8.15169829,  7.7966706 ],
         [ 8.52030967,  7.65428744,  8.75414826, ...,  7.82714759,
           8.42450845,  7.97391585],
...
         [24.97451368, 22.86439187, 26.31542397, ..., 22.32622279,
          23.37499846, 20.14747941],
         [25.72309773, 23.41361364, 26.26496159, ..., 22.53506766,
          24.17922795, 20.54121517],
         [25.39677988, 23.47981812, 26.41583286, ..., 22.64708166,
          24.18997302, 20.82430975]],

        [[25.63436137, 24.64937267, 23.02377228, ..., 21.38001881,
          23.28754127, 20.86465008],
         [25.71725938, 24.86083727, 23.32906441, ..., 21.53938202,
          23.75169282, 21.20184085],
         [25.8176766 , 24.79599903, 23.46932209, ..., 21.51348049,
          23.39116013, 21.32608358],
         ...,
         [25.51729876, 24.06698435, 26.26428369, ..., 22.83561348,
          24.35766575, 21.25306714],
         [25.82404414, 23.77358827, 26.40104669, ..., 22.81620418,
          24.29625325, 20.98455021],
         [25.23570924, 23.76638766, 26.33719882, ..., 22.48158114,
          24.33738386, 20.8663244 ]]]])
Coordinates:
  * time     (time) datetime64[ns] 2016-01-02 2016-01-03 ... 2016-07-31
  * number   (number) int32 0 1 2 3 4 5 6 7 8 9 ... 16 17 18 19 20 21 22 23 24
  * y        (y) float64 50.52 50.45 50.37 50.3 50.23 ... 43.27 43.2 43.13 43.05
  * x        (x) float64 3.715 3.822 3.93 4.038 ... 16.75 16.85 16.96 17.07

In [14]:
# calculate monthly means
y_pred_mm = y_pred.groupby('time.month').mean(dim=('time'))
y_true_mm = y_true.groupby('time.month').mean(dim=('time'))

In [15]:
# calculate mean annual cycle
y_pred_ac = y_pred_mm.mean(dim=('x', 'y'))
y_true_ac = y_true_mm.mean(dim=('x', 'y'))

In [16]:
# compute daily anomalies
def anomalies(ds, timescale='time.dayofyear', standard=False):
        # group dataset by time scale
        groups = ds.groupby(timescale).groups

        # compute anomalies over time
        anomalies = {}
        for time, time_scale in groups.items():
            with warnings.catch_warnings():
                warnings.simplefilter('ignore', category=RuntimeWarning)
                # anomaly = (x(t) - mean(x, t))
                anomalies[time] = (ds.isel(time=time_scale) -
                                   ds.isel(time=time_scale).mean(dim='time'))

                # standardized anomaly = (x(t) - mean(x, t)) / std(x, t)
                if standard:
                    anomalies[time] /= ds.isel(time=time_scale).std(dim='time')

        # concatenate anomalies and sort chronologically
        anomalies = xr.concat(anomalies.values(), dim='time')
        anomalies = anomalies.sortby(anomalies.time)

        return anomalies

y_pred_anom = anomalies(y_pred, timescale='time.month')
y_true_anom = anomalies(y_true, timescale='time.month')

In [17]:
y_pred_anom

<xarray.DataArray 'tasmean' (time: 212, y: 103, x: 125, number: 25)>
array([[[[ 1.87297319e+00,  1.18490792e+00,  6.52305540e+00, ...,
           2.74206827e+00,  2.70284900e+00,  4.56835708e+00],
         [ 1.97434283e+00,  1.29665036e+00,  6.70635505e+00, ...,
           2.82939517e+00,  2.65069911e+00,  4.62149654e+00],
         [ 1.82844160e+00,  1.24153823e+00,  6.61991326e+00, ...,
           2.85678787e+00,  2.73163177e+00,  4.82136422e+00],
         ...,
         [-4.24307357e+00, -5.51292019e+00,  1.21402643e+00, ...,
          -3.44196957e+00, -3.47139983e+00, -3.44563066e+00],
         [-4.07239502e+00, -5.22595248e+00,  1.79622125e+00, ...,
          -3.49342510e+00, -3.30246682e+00, -3.48282559e+00],
         [-5.17321228e+00, -5.90166078e+00,  1.46425137e+00, ...,
          -3.86772204e+00, -3.56227247e+00, -3.86426034e+00]],

        [[ 2.01521085e+00,  1.28614858e+00,  6.75113499e+00, ...,
           2.91100614e+00,  2.82531462e+00,  4.77790248e+00],
         [ 1.78482912e+00,  1.32799368e+00,  6.58009509e+00, ...,
           2.77273149e+00,  2.60715747e+00,  4.58172031e+00],
         [ 2.02911226e+00,  1.18487930e+00,  6.72429203e+00, ...,
           2.69805290e+00,  2.75080203e+00,  4.74353091e+00],
...
         [ 2.85826699e+00,  1.12954147e+00,  3.15979736e+00, ...,
          -1.73318809e+00, -1.09393135e+00, -2.11628483e+00],
         [ 3.11747374e+00,  1.30116813e+00,  2.67430893e+00, ...,
          -1.99238791e+00, -8.38477780e-01, -2.15365129e+00],
         [ 2.52277244e+00,  1.17652228e+00,  2.66914292e+00, ...,
          -1.90993273e+00, -8.00498483e-01, -2.04684002e+00]],

        [[ 4.41773802e+00,  2.69872183e+00,  2.67871331e+00, ...,
           1.46303803e+00, -2.16211015e-01,  2.37900813e-01],
         [ 4.26645841e+00,  2.66523796e+00,  2.69898170e+00, ...,
           1.32602953e+00, -7.04546462e-03,  2.43429267e-01],
         [ 4.36547424e+00,  2.63390509e+00,  2.88471858e+00, ...,
           1.28394994e+00, -2.36615041e-01,  3.77199647e-01],
         ...,
         [ 2.49533134e+00,  1.53421414e+00,  2.38479015e+00, ...,
          -1.81957713e+00, -5.98524796e-01, -1.95258672e+00],
         [ 2.75384098e+00,  1.24019981e+00,  2.48288272e+00, ...,
          -1.92196691e+00, -7.70089733e-01, -2.18916363e+00],
         [ 2.45012496e+00,  1.51384146e+00,  2.65071192e+00, ...,
          -2.00976399e+00, -4.69653114e-01, -2.09109621e+00]]]])
Coordinates:
  * time     (time) datetime64[ns] 2016-01-02 2016-01-03 ... 2016-07-31
  * number   (number) int32 0 1 2 3 4 5 6 7 8 9 ... 16 17 18 19 20 21 22 23 24
  * y        (y) float64 50.52 50.45 50.37 50.3 50.23 ... 43.27 43.2 43.13 43.05
  * x        (x) float64 3.715 3.822 3.93 4.038 ... 16.75 16.85 16.96 17.07

In [18]:
y_true_anom

<xarray.DataArray 'tasmean' (time: 212, y: 103, x: 125)>
array([[[ 3.4642730e+00,  3.4476929e+00,  3.3864155e+00, ...,
         -6.1212840e+00, -6.2449260e+00, -6.3466835e+00],
        [ 3.5148864e+00,  3.4460711e+00,  3.3987265e+00, ...,
         -5.9551573e+00, -6.0563812e+00, -6.1319847e+00],
        [ 3.4569592e+00,  3.3555593e+00,  3.3448315e+00, ...,
         -4.9077044e+00, -5.7809181e+00, -5.7981954e+00],
        ...,
        [ 1.9665375e+00,  2.0071964e+00,  2.0541601e+00, ...,
         -9.5330143e-01, -8.9785385e-01, -1.2276487e+00],
        [ 1.7831011e+00,  1.8191099e+00,  1.8840427e+00, ...,
         -1.3906078e+00, -1.4576063e+00, -1.3541088e+00],
        [ 1.5885782e+00,  1.6341772e+00,  1.6895132e+00, ...,
         -5.8984089e-01, -9.4870090e-01, -1.4086294e+00]],

       [[ 1.5652862e+00,  1.5699463e+00,  1.5545368e+00, ...,
         -1.0690619e+01, -1.0596123e+01, -1.0456272e+01],
        [ 1.5502868e+00,  1.5093951e+00,  1.5329428e+00, ...,
         -1.0485126e+01, -1.0586105e+01, -1.0606625e+01],
        [ 1.5154309e+00,  1.4783010e+00,  1.5372753e+00, ...,
         -9.4249468e+00, -1.0200474e+01, -1.0465218e+01],
...
        [-7.4304581e-02,  2.0864296e-01,  3.1406403e-01, ...,
          7.1717834e-01,  6.6141319e-01,  7.1131134e-01],
        [ 8.2895279e-02,  2.4258423e-01,  3.0492973e-01, ...,
          9.1749382e-01,  6.9561577e-01,  1.3314896e+00],
        [ 1.9221497e-01,  2.4340057e-01,  2.4814034e-01, ...,
          2.4197960e-01,  2.3776054e-01,  8.2266998e-01]],

       [[-9.7415352e-01, -1.1335812e+00, -1.1033554e+00, ...,
          1.8207417e+00,  1.5464363e+00,  1.3926964e+00],
        [-1.0945187e+00, -1.1030312e+00, -1.0566883e+00, ...,
          2.1242447e+00,  1.6871147e+00,  1.4030552e+00],
        [-1.2223682e+00, -1.1682606e+00, -1.0695095e+00, ...,
          2.4668903e+00,  1.9895363e+00,  1.6027279e+00],
        ...,
        [ 1.4066525e+00,  1.5557499e+00,  1.5779190e+00, ...,
          1.2552338e+00,  1.2683468e+00,  1.6657791e+00],
        [ 1.3363132e+00,  1.4913330e+00,  1.4936199e+00, ...,
          1.3025341e+00,  1.2614117e+00,  1.8465347e+00],
        [ 1.2342072e+00,  1.2735825e+00,  1.2895527e+00, ...,
          6.5552330e-01,  7.4807549e-01,  1.3280716e+00]]], dtype=float32)
Coordinates:
  * x                  (x) float64 3.715 3.822 3.93 4.038 ... 16.85 16.96 17.07
  * y                  (y) float64 50.52 50.45 50.37 50.3 ... 43.2 43.13 43.05
    heightAboveGround  float64 2.0
  * time               (time) datetime64[ns] 2016-01-02 ... 2016-07-31

### Coefficient of determination

In [19]:
"""
# get predicted and observed daily anomalies
y_pred_av_number = []
for n in range(y_pred_anom.number.size):
    y_pred_anom_number = y_pred_anom.sel(number=n)
    y_pred_anom_number = y_pred_anom_number.values.flatten()
    y_pred_av_number.append(y_pred_anom_number)

y_true_av = y_true_anom.values.flatten()

y_pred_mm_number = []
for n in range(y_pred_mm.number.size):
    y_pred_mm_number = y_pred_mm.sel(number=n)
    y_pred_mm_number = y_pred_mm_number.values.flatten()
    y_pred_mv_number.append(y_pred_anom_number)
    
"""

'\n# get predicted and observed daily anomalies\ny_pred_av_number = []\nfor n in range(y_pred_anom.number.size):\n    y_pred_anom_number = y_pred_anom.sel(number=n)\n    y_pred_anom_number = y_pred_anom_number.values.flatten()\n    y_pred_av_number.append(y_pred_anom_number)\n\ny_true_av = y_true_anom.values.flatten()\n\ny_pred_mm_number = []\nfor n in range(y_pred_mm.number.size):\n    y_pred_mm_number = y_pred_mm.sel(number=n)\n    y_pred_mm_number = y_pred_mm_number.values.flatten()\n    y_pred_mv_number.append(y_pred_anom_number)\n    \n'

In [20]:
def flatten(x):
    x_ds_in_array_appended = []
    for n in range(x.number.size):
        x_number = x.sel(number=n)
        x_number = x_number.values.flatten()
        x_ds_in_array_appended.append(x_number)    
    return x_ds_in_array_appended

y_pred_av = flatten(y_pred_anom)
y_pred_mv = flatten(y_pred_mm)

y_true_av = y_true_anom.values.flatten()
y_true_mv = y_true_mm.values.flatten()

r2_mm_appender = []
r2_anom_appender = []



In [21]:
y_true_av.shape

(2729500,)

In [22]:
for n in range(y_pred_anom.number.size):
    # apply mask of valid pixels
    y_pred_av_number = y_pred_av[n]
    mask = (~np.isnan(y_pred_av_number[n]) & ~np.isnan(y_true_av))
    y_pred_av_number = y_pred_av_number[mask]
    y_true_av = y_true_av[mask]

    # apply mask of valid pixels
    y_pred_mv_number = y_pred_mv[n]
    mask = (~np.isnan(y_pred_mv_number) & ~np.isnan(y_true_mv))
    y_pred_mv_number = y_pred_mv_number[mask]
    y_true_mv = y_true_mv[mask]

    # calculate coefficient of determination on monthly means
    r2_mm = r2_score(y_true_mv, y_pred_mv_number)
    print('R2 on monthly means: {:.2f}'.format(r2_mm))
    r2_mm_appender.append(r2_mm)

    # calculate coefficient of determination on daily anomalies
    r2_anom = r2_score(y_true_av, y_pred_av_number)
    print('R2 on daily anomalies: {:.2f}'.format(r2_anom))
    r2_anom_appender.append(r2_anom)

ValueError: Found input variables with inconsistent numbers of samples: [90125, 2729500]

### Coefficient of determination: Spatially

In [ ]:
# iterate over the grid points
r2 = np.ones((2, len(y_pred.x), len(y_pred.y)), dtype=np.float32) * np.nan
for i in range(len(y_pred.x)):
    for j in range(len(y_pred.y)):
        # get observed and predicted monthly mean temperature for current grid point
        point_true = y_true_mm.isel(x=i, y=j)
        point_pred = y_pred_mm.isel(x=i, y=j)

        # remove missing values
        mask = ((~np.isnan(point_true)) & (~np.isnan(point_pred)))
        point_true = point_true[mask].values
        point_pred = point_pred[mask].values
        if point_true.size < 1:
            continue
        
        # get anomalies for current grid point
        point_anom_true = y_true_anom.isel(x=i, y=j)
        point_anom_pred = y_pred_anom.isel(x=i, y=j)
        
        # remove missing values
        mask_anom = ((~np.isnan(point_anom_true)) & (~np.isnan(point_anom_pred)))
        point_anom_true = point_anom_true[mask_anom].values
        point_anom_pred = point_anom_pred[mask_anom].values

        # compute coefficient of determination
        r2[0, i, j] = r2_score(point_true, point_pred)
        r2[1, i, j] = r2_score(point_anom_true, point_anom_pred)

In [ ]:
# define color map: red to green
grn = cm.get_cmap('Greens', 128)
red = cm.get_cmap('Reds_r', 128)
red2green = ListedColormap(np.vstack((red(np.linspace(0, 1, 128)),
                                      grn(np.linspace(0, 1, 128)))))

# plot coefficient of determination
vmin, vmax = 0.5, 1
fig, ax = plt.subplots(1, 2, figsize=(10, 10))

# monthly means
im0 = ax[0].imshow(r2[0, :], origin='lower', cmap=red2green, vmin=vmin, vmax=vmax)
ax[0].text(x=r2.shape[1] - 2, y=2, s='Average R$^2$: {:.2f}'.format(np.nanmean(r2[0, :])), fontsize=14, ha='right');
ax[0].set_axis_off()
ax[0].set_title('Monthly mean {} (°C)'.format(NAMES[PREDICTAND]), fontsize=14, pad=10);

# daily anomalies
im1 = ax[1].imshow(r2[1, :], origin='lower', cmap=red2green, vmin=vmin, vmax=vmax)
ax[1].text(x=r2.shape[1] - 2, y=2, s='Average R$^2$: {:.2f}'.format(np.nanmean(r2[1, :])), fontsize=14, ha='right');
ax[1].set_axis_off()
ax[1].set_title('Daily {} anomaly (°C)'.format(NAMES[PREDICTAND]), fontsize=14, pad=10);

# add colorbar 
cbar_ax_bias = fig.add_axes([ax[1].get_position().x1 + 0.05, ax[1].get_position().y0,
                             0.03, ax[1].get_position().y1 - ax[1].get_position().y0])
cbar_bias = fig.colorbar(im0, cax=cbar_ax_bias)
cbar_bias.set_label(label='Coefficient of determination R$^2$', fontsize=14)
cbar_bias.ax.tick_params(labelsize=14, pad=10)

# save figure
fig.savefig('../Notebooks/Figures/{}_r2_spatial.png'.format(PREDICTAND), dpi=300, bbox_inches='tight')

### Bias

Calculate yearly average bias over entire reference period:

In [ ]:
# yearly average bias over reference period
y_pred_yearly_avg = y_pred.groupby('time.year').mean(dim='time')
y_true_yearly_avg = y_true.groupby('time.year').mean(dim='time')
bias_yearly_avg = y_pred_yearly_avg - y_true_yearly_avg
print('({}:{}) Yearly average bias of {}: {:.2f}°C'.format(TRUE, PRED, PREDICTAND, bias_yearly_avg.mean().item()))

In [ ]:
# mean absolute error over reference period
mae_avg = np.abs(y_pred_yearly_avg - y_true_yearly_avg).mean()
print('({}:{}) Yearly average MAE of {}: {:.2f}°C'.format(TRUE, PRED, PREDICTAND, mae_avg.mean().item()))

In [ ]:
# root mean squared error over reference period
rmse_avg = np.sqrt(((y_pred_yearly_avg - y_true_yearly_avg) ** 2).mean())
print('({}:{}) Yearly average RMSE of {}: {:.2f}°C'.format(TRUE, PRED, PREDICTAND, rmse_avg.mean().item()))

In [ ]:
# Pearson's correlation coefficient over reference period
correlations = []
for year in y_pred_yearly_avg.year:
    y_p = y_pred_yearly_avg.sel(year=year).values        
    y_t = y_true_yearly_avg.sel(year=year).values
    r, _ = stats.pearsonr(y_p[~np.isnan(y_p)], y_t[~np.isnan(y_t)])
    print('({:d}): {:.2f}'.format(year.item(), r))
    correlations.append(r)
print('Yearly average Pearson correlation coefficient for {}: {:.2f}'.format(PREDICTAND, np.asarray(r).mean()))

In [ ]:
# plot average of observation, prediction, and bias
vmin, vmax = (-15, 15) if PREDICTAND == 'tasmin' else (0, 25)
fig, axes = plt.subplots(1, 3, figsize=(24, 8), sharex=True, sharey=True)
axes = axes.flatten()
for ds, ax in zip([y_true_yearly_avg, y_pred_yearly_avg, bias_yearly_avg], axes):
    if ds is bias_yearly_avg:
        ds = ds.mean(dim='year')
        im2 = ax.imshow(ds.values, origin='upper', cmap='RdBu_r', vmin=-2, vmax=2)
        ax.text(x=ds.shape[0] - 2, y=2, s='Average: {:.2f}°C'.format(ds.mean().item()), fontsize=14, ha='right')
    else:
        im1 = ax.imshow(ds.mean(dim='year').values, origin='upper', cmap='RdYlBu_r', vmin=vmin, vmax=vmax)
        
# set titles
axes[0].set_title(TRUE, fontsize=16, pad=10);
axes[1].set_title(PRED, fontsize=16, pad=10);
axes[2].set_title('Bias', fontsize=16, pad=10);

# adjust axes
for ax in axes.flat:
    ax.axes.get_xaxis().set_ticklabels([])
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticklabels([])
    ax.axes.get_yaxis().set_ticks([])
    ax.axes.axis('tight')
    ax.set_xlabel('')
    ax.set_ylabel('')

# adjust figure
fig.suptitle('Average {}: {} - {}'.format(NAMES[PREDICTAND], start_year, end_year), fontsize=20);
fig.subplots_adjust(hspace=0, wspace=0, top=0.85)

# add colorbar for bias
axes = axes.flatten()
cbar_ax_bias = fig.add_axes([axes[-1].get_position().x1 + 0.01, axes[-1].get_position().y0,
                             0.01, axes[-1].get_position().y1 - axes[-1].get_position().y0])
cbar_bias = fig.colorbar(im2, cax=cbar_ax_bias)
cbar_bias.set_label(label='Bias / (°C)', fontsize=16)
cbar_bias.ax.tick_params(labelsize=14)

# add colorbar for predictand
cbar_ax_predictand = fig.add_axes([axes[0].get_position().x0, axes[0].get_position().y0 - 0.1,
                                   axes[-1].get_position().x0 - axes[0].get_position().x0,
                                   0.05])
cbar_predictand = fig.colorbar(im1, cax=cbar_ax_predictand, orientation='horizontal')
cbar_predictand.set_label(label='{} / (°C)'.format(NAMES[PREDICTAND].capitalize()), fontsize=16)
cbar_predictand.ax.tick_params(labelsize=14)

# add metrics: MAE and RMSE
axes[1].text(x=ds.shape[0] - 2, y=2, s='MAE = {:.2f}°C'.format(mae_avg.mean().item()), fontsize=14, ha='right')
axes[1].text(x=ds.shape[0] - 2, y=12, s='RMSE = {:.2f}°C'.format(rmse_avg.mean().item()), fontsize=14, ha='right')

# save figure
fig.savefig('../Notebooks/Figures/{}_average_bias.png'.format(PREDICTAND), dpi=300, bbox_inches='tight')

### Seasonal bias

Calculate seasonal bias:

In [ ]:
# group data by season: (DJF, MAM, JJA, SON)
y_true_snl = y_true.groupby('time.season').mean(dim='time')
y_pred_snl = y_pred.groupby('time.season').mean(dim='time')
bias_snl = y_pred_snl - y_true_snl

In [ ]:
# print average bias per season: Insitu:CERRA
for season in bias_snl.season:
    print('({}:{}) Average bias of mean {} for season {}: {:.1f}°C'.format(TRUE, PRED, PREDICTAND, season.item(), bias_snl.sel(season=season).mean().item()))

Plot seasonal differences, taken from the [xarray documentation](xarray.pydata.org/en/stable/examples/monthly-means.html).

In [ ]:
# plot seasonal differences
seasons = ('DJF', 'JJA')
fig, axes = plt.subplots(nrows=1, ncols=len(seasons) + 1, figsize=(24, 8), sharex=True, sharey=True)
axes = axes.flatten()

# plot annual average bias
ds = bias_yearly_avg.mean(dim='year')
im = axes[0].imshow(ds.values, origin='lower', cmap='RdBu_r', vmin=-2, vmax=2)
axes[0].set_title('Annual', fontsize=16);
axes[0].text(x=ds.shape[0] - 2, y=2, s='Average: {:.2f}°C'.format(ds.mean().item()), fontsize=14, ha='right')

# plot seasonal average bias
for ax, season in zip(axes[1:], seasons):
    ds = bias_snl.sel(season=season)
    ax.imshow(ds.values, origin='lower', cmap='RdBu_r', vmin=-2, vmax=2)
    ax.set_title(season, fontsize=16);
    ax.text(x=ds.shape[0] - 2, y=2, s='Average: {:.2f}°C'.format(ds.mean().item()), fontsize=14, ha='right')

# adjust axes
for ax in axes.flat:
    ax.axes.get_xaxis().set_ticklabels([])
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticklabels([])
    ax.axes.get_yaxis().set_ticks([])
    ax.axes.axis('tight')
    ax.set_xlabel('')
    ax.set_ylabel('')

# adjust figure
fig.suptitle('Average bias of {}: {} - {}'.format(NAMES[PREDICTAND], start_year, end_year), fontsize=20);
fig.subplots_adjust(hspace=0, wspace=0, top=0.85)

# add colorbar
cbar_ax_predictand = fig.add_axes([axes[-1].get_position().x1 + 0.01, axes[-1].get_position().y0,
                                   0.01, axes[0].get_position().y1 - axes[-1].get_position().y0])
cbar_predictand = fig.colorbar(im, cax=cbar_ax_predictand)
cbar_predictand.set_label(label='Bias / (°C)', fontsize=16)
cbar_predictand.ax.tick_params(labelsize=14)

# save figure
fig.savefig('../Notebooks/Figures/{}_bias_seasonal.png'.format(PREDICTAND), dpi=300, bbox_inches='tight')

### Bias of extreme values

In [ ]:
# extreme quantile of interest
quantile = 0.02 if PREDICTAND == 'tasmin' else 0.98

In [ ]:
# calculate extreme quantile for each year
with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=RuntimeWarning)
    y_pred_ex = y_pred.groupby('time.year').quantile(quantile, dim='time')
    y_true_ex = y_true.groupby('time.year').quantile(quantile, dim='time')

In [ ]:
# calculate bias in extreme quantile for each year
bias_ex = y_pred_ex - y_true_ex

In [ ]:
# bias of extreme quantile: Insitu:CERRA
print('({}:{}) Yearly average bias for P{:.0f} of {}: {:.1f}°C'.format(TRUE, PRED, quantile * 100, PREDICTAND, bias_ex.mean().item()))

In [ ]:
# mean absolute error in extreme quantile
mae_ex = np.abs(y_pred_ex - y_true_ex).mean()

In [ ]:
# mae of extreme quantile: Insitu:CERRA
print('({}:{}) Yearly average MAE for P{:.0f} of {}: {:.1f}°C'.format(TRUE, PRED, quantile * 100, PREDICTAND, mae_ex.item()))

In [ ]:
# root mean squared error in extreme quantile
rmse_ex = np.sqrt(((y_pred_ex - y_true_ex) ** 2).mean())

In [ ]:
# rmse of extreme quantile: Insitu:CERRA
print('({}:{}) Yearly average RMSE for P{:.0f} of {}: {:.1f}°C'.format(TRUE, PRED, quantile * 100, PREDICTAND, rmse_ex.item()))

In [ ]:
# plot extremes of observation, prediction, and bias
vmin, vmax = (-20, 0) if PREDICTAND == 'tasmin' else (10, 40)
fig, axes = plt.subplots(1, 3, figsize=(24, 8), sharex=True, sharey=True)
axes = axes.flatten()
for ds, ax in zip([y_true_ex, y_pred_ex, bias_ex], axes):
    if ds is bias_ex:
        ds = ds.mean(dim='year')
        im2 = ax.imshow(ds.values, origin='upper', cmap='RdBu_r', vmin=-2, vmax=2)
        ax.text(x=ds.shape[0] - 2, y=2, s='Average: {:.2f}°C'.format(ds.mean().item()), fontsize=14, ha='right')
    else:
        im1 = ax.imshow(ds.mean(dim='year').values, origin='upper', cmap='Blues_r' if PREDICTAND == 'tasmin' else 'Reds',
                        vmin=vmin, vmax=vmax)

# set titles
axes[0].set_title(TRUE, fontsize=16, pad=10);
axes[1].set_title(PRED, fontsize=16, pad=10);
axes[2].set_title('Bias', fontsize=16, pad=10);

# adjust axes
for ax in axes.flat:
    ax.axes.get_xaxis().set_ticklabels([])
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticklabels([])
    ax.axes.get_yaxis().set_ticks([])
    ax.axes.axis('tight')
    ax.set_xlabel('')
    ax.set_ylabel('')

# adjust figure
fig.suptitle('Average P{:.0f} of {}: {} - {}'.format(quantile * 100, NAMES[PREDICTAND], start_year, end_year), fontsize=20);
fig.subplots_adjust(hspace=0, wspace=0, top=0.85)

# add colorbar for bias
axes = axes.flatten()
cbar_ax = fig.add_axes([axes[-1].get_position().x1 + 0.01, axes[-1].get_position().y0,
                        0.01, axes[-1].get_position().y1 - axes[-1].get_position().y0])
cbar = fig.colorbar(im2, cax=cbar_ax)
cbar.set_label(label='Bias / (°C)', fontsize=16)
cbar.ax.tick_params(labelsize=14)

# add colorbar for predictand
cbar_ax_predictand = fig.add_axes([axes[0].get_position().x0, axes[0].get_position().y0 - 0.1,
                                   axes[-1].get_position().x0 - axes[0].get_position().x0,
                                   0.05])
cbar_predictand = fig.colorbar(im1, cax=cbar_ax_predictand, orientation='horizontal')
cbar_predictand.set_label(label='{} / (°C)'.format(NAMES[PREDICTAND].capitalize()), fontsize=16)
cbar_predictand.ax.tick_params(labelsize=14)

# add metrics: MAE and RMSE
axes[1].text(x=ds.shape[0] - 2, y=2, s='MAE = {:.2f}°C'.format(mae_ex.item()), fontsize=14, ha='right')
axes[1].text(x=ds.shape[0] - 2, y=12, s='RMSE = {:.2f}°C$^2$'.format(rmse_ex.item()), fontsize=14, ha='right')

# save figure
fig.savefig('../Notebooks/Figures/{}_bias_p{:.0f}.png'.format(PREDICTAND, quantile * 100), dpi=300, bbox_inches='tight')

In [ ]:
# scatter plot of observations vs. predictions
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# plot entire dataset
ax.plot(y_true_mv, y_pred_mv, 'o', alpha=.5, markeredgecolor='grey', markerfacecolor='none', markersize=3)

# plot 1:1 mapping line
if PREDICTAND == 'tasmin':
    interval = np.arange(-25, 30, 5)
else:
    interval = np.arange(-15, 45, 5)

ax.plot(interval, interval, color='k', lw=2, ls='--')

# add coefficients of determination
ax.text(interval[-1] - 0.5, interval[0] + 0.5, s='R$^2$ (monthly means)= {:.2f}'.format(r2_mm), ha='right', fontsize=18)
ax.text(interval[-1] - 0.5, interval[0] + 2.5, s='R$^2$ (daily anomalies) = {:.2f}'.format(r2_anom), ha='right', fontsize=18)
ax.text(interval[-1] - 0.5, interval[0] + 4.5, s='Yearly average bias of {}: {:.2f}°C'.format(PREDICTAND, bias_yearly_avg.mean().item()), ha='right', fontsize=18)
ax.text(interval[-1] - 0.5, interval[0] + 6.5, s='Yearly average MAE of {}: {:.2f}°C'.format(PREDICTAND, mae_avg.mean().item()), ha='right', fontsize=18)
ax.text(interval[-1] - 0.5, interval[0] + 8.5, s='Yearly average RMSE of {}: {:.2f}°C'.format(PREDICTAND, rmse_avg.mean().item()), ha='right', fontsize=18)


# format axes
ax.set_ylim(interval[0], interval[-1])
ax.set_xlim(interval[0], interval[-1])
ax.set_xticks(interval)
ax.set_xticklabels(interval, fontsize=16)
ax.set_yticks(interval)
ax.set_yticklabels(interval, fontsize=16)
ax.set_xlabel('CERRA_tasmean', fontsize=18)
ax.set_ylabel(PRED, fontsize=18)
ax.set_title('Monthly mean {} (°C)'.format(NAMES[PREDICTAND]), fontsize=20, pad=10);

# add axis for annual cycle
axins = inset_axes(ax, width="35%", height="40%", loc=2, borderpad=1)
axins.plot(y_pred_ac.values, ls='-', color='r', label=PRED)
axins.plot(y_true_ac.values, ls='--', color='k', label='CERRA')
axins.legend(frameon=False, fontsize=12, loc='lower center');
axins.yaxis.tick_right()
axins.set_yticks(np.arange(-10, 11, 2) if PREDICTAND == 'tasmin' else np.arange(0, 20, 2))
axins.set_yticklabels(np.arange(-10, 11, 2) if PREDICTAND == 'tasmin' else np.arange(0, 20, 2), fontsize=12)
axins.set_xticks(np.arange(0, 12))
axins.set_xticklabels([calendar.month_name[i + 1] for i in np.arange(0, 12)], rotation=90, fontsize=12)
axins.set_title('Mean annual cycle', fontsize=14, pad=5);

plt.savefig('mslp.png', dpi=300, bbox_inches='tight')
